In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [57]:
inaugural_speeches = pd.DataFrame(columns=["date", "name", "text", "link"])
inauglinks = []
base_url = "https://www.presidency.ucsb.edu/documents/"
# URL of the page to scrape
url = "https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/inaugural-addresses"

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the links on the page
links = soup.find_all("a")


for link in links:
    if "inaugural-address-" in str(link) or "the-presidents-inaugural-address" in str(link) or "https://www.presidency.ucsb.edu/documents/inaugural-address" in str(link):
        url = link.get("href")
        date = link.text
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        text = soup.find("div", class_="field-docs-content").find_all("p")
        text = [t.text for t in text]
        text = " ".join(text)
        links2 = soup.find_all("a")
        for link in links2:
            if "people" in str(link):
                person = str(link.get("href")).split("-")[-1]
        inaugural_speeches.loc[len(inaugural_speeches.index)] = [date, person, text, url]


inaugural_speeches

,date,name,text,link
0,"April 30, 1789",washington,Fellow-Citizens of the Senate and of the House...,https://www.presidency.ucsb.edu/documents/inau...
1,"March 4, 1793",washington,Fellow Citizens: I AM again called upon by the...,https://www.presidency.ucsb.edu/documents/inau...
2,"March 4, 1797",adams,"WHEN it was first perceived, in early times, t...",https://www.presidency.ucsb.edu/documents/inau...
3,"March 4, 1801",jefferson,Friends and Fellow-Citizens: CALLED upon to un...,https://www.presidency.ucsb.edu/documents/inau...
4,"March 4, 1805",jefferson,"PROCEEDING, fellow-citizens, to that qualifica...",https://www.presidency.ucsb.edu/documents/inau...
5,"March 4, 1809",madison,Unwilling to depart from examples of the most ...,https://www.presidency.ucsb.edu/documents/inau...
6,"March 4, 1813",madison,About to add the solemnity of an oath to the o...,https://www.presidency.ucsb.edu/documents/inau...
7,"March 4, 1817",monroe,I should be destitute of feeling if I was not ...,https://www.presidency.ucsb.edu/documents/inau...
8,"March 4, 1821",monroe,Fellow-Citizens: I shall not attempt to descri...,https://www.presidency.ucsb.edu/documents/inau...
9,"March 4, 1825",adams,In compliance with an usage coeval with the ex...,https://www.presidency.ucsb.edu/documents/inau...


In [58]:
inaugural_speeches['text'][1]

'Fellow Citizens: I AM again called upon by the voice of my country to execute the functions of its Chief Magistrate. When the occasion proper for it shall arrive, I shall endeavor to express the high sense I entertain of this distinguished honor, and of the confidence which has been reposed in me by the people of united America. Previous to the execution of any official act of the President the Constitution requires an oath of office. This oath I am now about to take, and in your presence: That if it shall be found during my administration of the Government I have in any instance violated willingly or knowingly the injunctions thereof, I may (besides incurring constitutional punishment) be subject to the upbraidings of all who are now witnesses of the present solemn ceremony.'

In [59]:
inaugural_speeches = inaugural_speeches.drop(index=range(41))
inaugural_speeches


,date,name,text,link
41,"January 20, 1961",kennedy,"Vice President Johnson, Mr. Speaker, Mr. Chief...",https://www.presidency.ucsb.edu/documents/inau...
42,"January 20, 1965",johnson,My fellow countrymen: On this occasion the oat...,https://www.presidency.ucsb.edu/documents/the-...
43,"January 20, 1969",nixon,"Senator Dirksen, Mr. Chief Justice, Mr. Vice p...",https://www.presidency.ucsb.edu/documents/inau...
44,"January 20, 1977",carter,"For myself and for our Nation, I want to thank...",https://www.presidency.ucsb.edu/documents/inau...
45,"January 20, 1981",reagan,"Senator Hatfield, Mr. Chief Justice, Mr. Presi...",https://www.presidency.ucsb.edu/documents/inau...
46,"January 21, 1985",reagan,"Senator Mathias, Chief Justice Burger, Vice Pr...",https://www.presidency.ucsb.edu/documents/inau...
47,"January 20, 1989",bush,"Mr. Chief Justice, Mr. President, Vice Preside...",https://www.presidency.ucsb.edu/documents/inau...
48,"January 20, 1993",clinton,"My fellow citizens, today we celebrate the mys...",https://www.presidency.ucsb.edu/documents/inau...
49,"January 20, 1997",clinton,"My fellow citizens, at this last Presidential ...",https://www.presidency.ucsb.edu/documents/inau...
50,"January 20, 2001",bush,"Thank you, all. Chief Justice Rehnquist, Presi...",https://www.presidency.ucsb.edu/documents/inau...


In [60]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import string
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer

[nltk_data] Downloading package wordnet to /Users/nathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to /Users/nathan/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nathan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [61]:
inaugural_speeches['text_lower'] = ["".join(item for item in lst).lower() for lst in inaugural_speeches['text']]

In [64]:
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def preprocessing(text, tokenizer, stopwords, lemmatizer):
    # Make everything in the df["Text"] column into a lower-case string
    #text = ["".join(item for item in lst).lower() for lst in text]

    # Tokenize
    tokens = tokenizer.tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    
    return tokens

In [65]:
inaugural_speeches['tokens'] = inaugural_speeches['text_lower'].apply(lambda x: preprocessing(x, tokenizer, stopwords, lemmatizer))
inaugural_speeches

,date,name,text,link,text_lower,tokens
41,"January 20, 1961",kennedy,"Vice President Johnson, Mr. Speaker, Mr. Chief...",https://www.presidency.ucsb.edu/documents/inau...,"vice president johnson, mr. speaker, mr. chief...","[vice, president, johnson, speaker, chief, jus..."
42,"January 20, 1965",johnson,My fellow countrymen: On this occasion the oat...,https://www.presidency.ucsb.edu/documents/the-...,my fellow countrymen: on this occasion the oat...,"[fellow, countryman, occasion, oath, taken, go..."
43,"January 20, 1969",nixon,"Senator Dirksen, Mr. Chief Justice, Mr. Vice p...",https://www.presidency.ucsb.edu/documents/inau...,"senator dirksen, mr. chief justice, mr. vice p...","[senator, dirksen, chief, justice, vice, presi..."
44,"January 20, 1977",carter,"For myself and for our Nation, I want to thank...",https://www.presidency.ucsb.edu/documents/inau...,"for myself and for our nation, i want to thank...","[nation, want, thank, predecessor, done, heal,..."
45,"January 20, 1981",reagan,"Senator Hatfield, Mr. Chief Justice, Mr. Presi...",https://www.presidency.ucsb.edu/documents/inau...,"senator hatfield, mr. chief justice, mr. presi...","[senator, hatfield, chief, justice, president,..."
46,"January 21, 1985",reagan,"Senator Mathias, Chief Justice Burger, Vice Pr...",https://www.presidency.ucsb.edu/documents/inau...,"senator mathias, chief justice burger, vice pr...","[senator, mathias, chief, justice, burger, vic..."
47,"January 20, 1989",bush,"Mr. Chief Justice, Mr. President, Vice Preside...",https://www.presidency.ucsb.edu/documents/inau...,"mr. chief justice, mr. president, vice preside...","[chief, justice, president, vice, president, q..."
48,"January 20, 1993",clinton,"My fellow citizens, today we celebrate the mys...",https://www.presidency.ucsb.edu/documents/inau...,"my fellow citizens, today we celebrate the mys...","[fellow, citizen, today, celebrate, mystery, a..."
49,"January 20, 1997",clinton,"My fellow citizens, at this last Presidential ...",https://www.presidency.ucsb.edu/documents/inau...,"my fellow citizens, at this last presidential ...","[fellow, citizen, last, presidential, inaugura..."
50,"January 20, 2001",bush,"Thank you, all. Chief Justice Rehnquist, Presi...",https://www.presidency.ucsb.edu/documents/inau...,"thank you, all. chief justice rehnquist, presi...","[thank, chief, justice, rehnquist, president, ..."


In [69]:
inaugural_speeches.reset_index(drop=True, inplace=True)


In [70]:
inaugural_speeches['tokens'][0]

['vice',
 'president',
 'johnson',
 'speaker',
 'chief',
 'justice',
 'president',
 'eisenhower',
 'vice',
 'president',
 'nixon',
 'president',
 'truman',
 'reverend',
 'clergy',
 'fellow',
 'citizen',
 'observe',
 'today',
 'victory',
 'party',
 'celebration',
 'freedom',
 'symbolizing',
 'end',
 'well',
 'beginning',
 'signifying',
 'renewal',
 'well',
 'change',
 'sworn',
 'almighty',
 'god',
 'solemn',
 'oath',
 'forebear',
 'prescribed',
 'nearly',
 'century',
 'three',
 'quarter',
 'ago',
 'world',
 'different',
 'man',
 'hold',
 'mortal',
 'hand',
 'power',
 'abolish',
 'form',
 'human',
 'poverty',
 'form',
 'human',
 'life',
 'yet',
 'revolutionary',
 'belief',
 'forebear',
 'fought',
 'still',
 'issue',
 'around',
 'globe',
 'belief',
 'right',
 'man',
 'come',
 'generosity',
 'state',
 'hand',
 'god',
 'dare',
 'forget',
 'today',
 'heir',
 'first',
 'revolution',
 'let',
 'word',
 'forth',
 'time',
 'place',
 'friend',
 'foe',
 'alike',
 'torch',
 'passed',
 'new',
 'gener